# <a name="top"></a>SWPLA PDDA 2020 Synthetic Sonic Log Generation Contest

**Author:** Ben Dowdell

**Date:** March XX, 2020

**Purpose:** To minimize RMSE in predicting Sonic logs (compressional & shear) from a suite of standard well logs

**Outline**

* [1. Initial Set-Up](#initial-setup)
** [1a. Standard Imports](#standard-imports)
** [1b. Sklearn Imports](#sklearn-imports)
** [1c. Helper Function Definitions](#helper-funcs)
* [2. Read Data](#read-data)
* [3. Inspect Data](#inspect-data)
* [4. EDA](#eda)
* [5. Feature Engineering](#fe)
** [5a. PE estimation](#fe-pe)
** [5b. DTC approximation](#fe-dtc)
** [5c. DTS approximation](#fe-dts)

### Data Decription
#### Files
#### train.csv
All the values equals to -999 are marked as missing values.
- CAL - Caliper, unit in Inch,  
- CNC - Neutron, unit in dec 
- GR - Gamma Ray, unit in API
- HRD - Deep Resisitivity, unit in Ohm per meter,
- HRM - Medium Resistivity, unit in Ohm per meter,
- PE - Photo-electric Factor, unit in Barn,
- ZDEN - Density, unit in Gram per cubit meter, 
- DTC - Compressional Travel-time, unit in nanosecond per foot,
- DTS - Shear Travel-time, unit in nanosecond per foot,


#### test.csv
The test data has all features that you used in the train dataset, except the two sonic curves DTC and DTS.

####  sample_submission.csv
A valid sample submission.
<p><font style="">
</font></p>

## <a name="initial-setup"></a>1. Initial Set-Up

Back to [top](#top)

### <a name="standard-imports"></a> 1a. Standard Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from pprint import pprint

random_state = 42
rng = np.random.RandomState(seed=42)

%matplotlib inline

### <a name="sklearn-imports"></a>1b. Sklearn Imports

In [ ]:
# Iterative Imputer for missing values
#from sklearn.experimental import enable_iterative_imputer
#from sklearn.impute import IterativeImputer

# Preprocessing utilities
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split

# Hyperparameter Tuning
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

# Regression Models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Evaluation Metrics
from sklearn.metrics import mean_squared_error, r2_score

# Visualization
from yellowbrick.model_selection import FeatureImportances

### <a name="helper-funcs"></a>1c. Helper Function Definitions

Helper function to plot data histograms

In [ ]:
def plot_data_distributions(df):
    """Plot histograms of each curve
    
    Paramters:
    df (pandas.DataFrame) : Input data frame containing log curves
    
    Returns:
    matplotlib.figure.Figure
    """
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(18,8))
    for ax, col in zip(axes.flatten(), df.columns.tolist()):
        ax.hist(df_w1[col])
        if 'HR' in col:
            ax.set_xscale('log')
        ax.set_title('{} Histogram'.format(col))
    plt.tight_layout()
    return fig

Helper function plot data CDF's

In [ ]:
def plot_data_cdf(df):
    """Plot cumulative distribution function of each curve
    
    Parameters:
    df (pandas.DataFrame) : Input data frame containing log curves
    
    Returns:
    matplotlib.figure.Figure
    """
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(18,8))
    for ax, col in zip(axes.flatten(), df.columns.tolist()):
        ax.hist(df_w1[col], density=True, cumulative=-1)
        if 'HR' in col:
            ax.set_xscale('log')
        ax.set_title('{} Histogram'.format(col))
    plt.tight_layout()
    return fig

Helper function to plot well curves in a normal log display

In [ ]:
def plot_well_curves(data):
    """
    Plots well log curves from an input data frame containing the data
    
    Parameters:
    data (pandas.DataFrame) : Input data frame containing the well log curves, one per column
    
    curve_names (list) : A list containing the column name of each well log curve in the input data frame
    
    Returns:
    matplotlib.figure.Figure
    """
    # get the column names as a list
    curve_names = data.columns.tolist()
    
    # create the figure
    fig, axes = plt.subplots(nrows=1, ncols=len(curve_names), sharey=True, figsize=(20,10))
    fig.suptitle('Training Well Data', fontsize=20)
    for ax, curve in zip(axes, curve_names):
        if curve in ['HRD', 'HRM']:
            ax.semilogx(data[curve], data.index, color='k')
        else:
            ax.plot(data[curve], data.index, color='k')
        if curve == 'CNC':
            ax.set_xlim(0.0, 1.0)
        if curve in ['DTC', 'DTS'] or 'DT' in curve:
            ax.set_title(curve, fontdict={'color':'r'})
            ax.invert_xaxis()
        else:
            ax.set_title(curve)
        ax.xaxis.tick_top()
        ax.invert_yaxis()
    fig.text(0.04, 0.5, 'Sample', va='center', rotation='vertical', fontdict={'fontsize':20})
    return fig

Helper function to plot Short Time Fourier Transform of the well curves (experimental)

In [ ]:
def plot_stft_well_curves(data):
    """
    Plots well log curves from an input data frame containing the data
    Overlays the well log curve on top of a STFT representation of the signal
    
    Parameters:
    data (pandas.DataFrame) : Input data frame containing the well log curves, one per column
    
    Returns:
    matplotlib.figure.Figure
    """
    # get the column names as a list
    curve_names = data.columns.tolist()
    
    # create the figure
    fig, axes = plt.subplots(nrows=1, ncols=len(curve_names), sharey=True, figsize=(20,10))
    fig.suptitle('Training Well Data', fontsize=20)
    for ax, curve in zip(axes, curve_names):
        f, t, Zxx = signal.stft(data[curve], fs=1)
        ax.pcolormesh(f.T, t.T, np.abs(Zxx).T, vmin=0, vmax=0.5)
        if curve in ['HRD', 'HRM']:
            ax1 = ax.twiny()
            ax1.semilogx(data[curve], data.index, color='k')
        else:
            ax1 = ax.twiny()
            ax1.plot(data[curve], data.index, color='k')
        if curve == 'CNC':
            ax1.set_xlim(0.0, 1.0)
        if curve in ['DTC', 'DTS']:
            ax.invert_xaxis()
            ax1.set_title(curve, fontdict={'color':'r'})
            ax1.invert_xaxis()
        else:
            ax.set_title(curve)
        ax1.xaxis.tick_top()
        ax.invert_yaxis()
    fig.text(0.04, 0.5, 'Sample', va='center', rotation='vertical', fontdict={'fontsize':20})
    return fig

Helper function to remove outliers from log curves using standard deviation from median value in a window

In [ ]:
def filter_curves(data, window, n_std):
    """
    Takes a data frame containing well log curves and filters outliers based on n-standard deviations
    from a median filtered version of the data.
    
    Parameters:
    data (pandas.DataFrame) : The input data frame containing data to be filtered.  Each column is a well log curve.
    
    window (int) : The size of the window to use in creating a median filtered curve (recommend 33)
    
    n_std (int) : The number of +/- standard deviations to use in considering outliers (recommend 2)
    
    Returns:
    df_clean (pandas.DataFrame) : The filtered well curves
    df_outliers (pandas.DataFrame) : The outlier data points removed by the filtering operation
    """
    # create a copy of the original data
    df_copy = data.copy()
    
    # create a data frame containing median-filtered version of the data
    df_medfilter = pd.DataFrame()
    for col in df_copy.columns.tolist():
        df_medfilter[col] = df_copy[col].rolling(window, center=True).median()
        
    # create a data frame containing standard deviation of the data
    df_stddev = pd.DataFrame()
    for col in df_copy.columns.tolist():
        df_stddev[col] = df_medfilter[col].rolling(window, center=True).std()
        
    # create a data frame containing the cleaned version of the data using standard deviation from the median filtered data
    # and create a data frame containing the removed outliers
    df_clean = pd.DataFrame()
    df_outliers = pd.DataFrame()
    for col in df_copy.columns.tolist():
        upper = df_medfilter[col] + df_stddev[col]*n_std
        lower = df_medfilter[col] - df_stddev[col]*n_std
        df_clean[col] = df_copy[col].where((df_copy[col] <= upper) & (df_copy[col] >= lower))
        #df_clean[col] = df_clean[col].interpolate(limit_area='inside')
        df_outliers[col] = df_copy[col].where(df_clean[col] != df_copy[col])
    
    return df_clean, df_outliers

Helper function to compare raw versus filtered logs

In [ ]:
def qc_curve_filt(data_raw, data_cleaned, *args):
    """
    Plots well log curves from an input data frame containing the data
    
    Parameters:
    data_raw (pandas.DataFrame) : Input data frame containing the well log curves, one log per column
    
    data_cleaned (pandas.DataFrame) : Input data frame containing filtered log curves, one log per column
    
    *args (pandas.DataFrame) : Optional data frame containing outliers
    
    Returns:
    matplotlib.figure.Figure
    """
    
    # get the column names as a list, assumes col names are the same for each input data frame
    curve_names = data_raw.columns.tolist()
    
    # build the figure
    fig, axes = plt.subplots(nrows=1, ncols=len(curve_names), sharey=True, figsize=(20,10))
    fig.suptitle('Training Well Data', fontsize=20)
    for ax, curve in zip(axes, curve_names):
        if curve in ['HRD', 'HRM']:
            ax.semilogx(data_raw[curve], data_raw.index, color='k', lw=3)
            ax.semilogx(data_cleaned[curve], data_cleaned.index, color='r', lw=1)
            for arg in args:
                ax.semilogx(arg[curve], arg.index, lw=0, marker='.', mfc='y', mec='k', alpha=0.2)
        else:
            ax.plot(data_raw[curve], data_raw.index, color='k', lw=3)
            ax.plot(data_cleaned[curve], data_cleaned.index, color='r', lw=1)
            for arg in args:
                ax.plot(arg[curve], arg.index, lw=0, marker='.', mfc='y', mec='k', alpha=0.2)
        if curve == 'CNC':
            ax.set_xlim(0.0, 1.0)
        if curve in ['DTC', 'DTS']:
            ax.set_title(curve, fontdict={'color':'r'})
            ax.invert_xaxis()
        else:
            ax.set_title(curve)
        ax.xaxis.tick_top()
        ax.invert_yaxis()
    fig.text(0.04, 0.5, 'Sample', va='center', rotation='vertical', fontdict={'fontsize':20})
    return fig

Helper function to compare individual predictors to DTC & DTS

In [ ]:
def compare_curves(predictor, dtc, dts):
    """
    Takes a predictor curve and plots it against both response variables, DTC & DTS
    
    Parameters:
    predictor (pandas.core.series.Series) : Independent Variable to compare, assumes either a column from a pandas DataFrame or ndarray-like
    
    dtc (pandas.core.series.Series) : DTC curve, assumes either a column from a pandas DataFrame or ndarray-like
    
    dts (pandas.core.series.Series) : DTS curve, assumes either a column from a pandas DataFrame or ndarray-like
    
    Returns:
    matplotlib.figure.Figure
    """
    fig = plt.figure(figsize=(20,10), linewidth=5, edgecolor='k')
    name = predictor.name
    fig.suptitle('{} curve comparison to DTC & DTS'.format(name), fontsize=20)
    
    ax1 = fig.add_subplot(2,2,1)
    if name == 'HRD':
        ax1.semilogy(predictor.index, predictor.values, 'k', lw=3)
    else:
        ax1.plot(predictor.index, predictor.values, 'k', lw=3)
    ax1c = ax1.twinx()
    ax1c.plot(dtc.index, dtc.values, 'r', lw=2)
    ax1c.invert_yaxis()
    ax1.set_ylabel(name)
    ax1c.set_ylabel('DTC', fontdict={'color':'r'})
    
    ax2 = fig.add_subplot(2,2,2)
    ax2 = sns.scatterplot(x=predictor, y=dtc)
    if name == 'HRD':
        ax2.set_xscale('log')
    ax2.set_xlabel(name)
    ax2.set_ylabel('DTC')
    
    ax3 = fig.add_subplot(2,2,3)
    if name == 'HRD':
        ax3.semilogy(predictor.index, predictor.values, 'k', lw=3)
    else:
        ax3.plot(predictor.index, predictor.values, 'k', lw=3)
    ax3c = ax3.twinx()
    ax3c.plot(dts.index, dts.values, 'r', lw=2)
    ax3c.invert_yaxis()
    ax3.set_ylabel(name)
    ax3c.set_ylabel('DTS', fontdict={'color':'r'})
    
    ax4 = fig.add_subplot(2,2,4)
    ax4 = sns.scatterplot(x=predictor, y=dts)
    if name == 'HRD':
        ax4.set_xscale('log')
    ax4.set_xlabel(name)
    ax4.set_ylabel('DTS')
    
    return fig

Helper function to calculate Root Mean Squared Error (RMSE), the primary evaluation metric for this project.

In [ ]:
def evaluate_rmse(real, predicted):
    """Calculates the Root Mean Square Error
     
     Parameters:
     real (ndarray) : actual values as a numpy array
     predicted (ndarray) : predicted values as a numpy array
     
     Returns: RMSE Accuracy as a float
     
    """
    mse = np.square(np.subtract(real, predicted)).mean()
    return np.sqrt(mse)

Helper function to plot logs in a typical petrophysical layout

In [ ]:
def petro_plot(gr, resd, neut, den, pe, dtc, dts):
    """
    Produces a typical petrophysical plot
    
    Parameters:
    gr (pandas.Series) : Gamma Ray curve as a pandas series
    resd (pandas.Series) : Deep Resistivity curve as a pandas series
    neut (pandas.Series) : Neutron Porosity curve as a pandas series
    den (pandas.Series) : Density curve as a pandas series
    pe (pandas.Series) : PE curve as a pandas series
    dtc (pandas.Series) : Compressional sonic curve as a pandas series
    dts (pandas.Series) : Shear sonic curve as a pandas series
    
    Returns : matplotlib.figure.Figure
    """
    
    fig = plt.figure(figsize=(20,12))

    # GR
    ax1 = fig.add_subplot(1,5,1)
    ax1.plot(gr, gr.index, 'k', lw=1)
    ax1.invert_yaxis()
    ax1.set_xlabel('GR')
    ax1.set_ylabel('Sample no.')
    ax1.xaxis.tick_top()
    ax1.xaxis.set_label_position('top')

    # RD
    ax2 = fig.add_subplot(1,5,2)
    ax2.semilogx(resd, resd.index, 'b', lw=1)
    ax2.set_xlabel('HRD')
    ax2.invert_yaxis()
    ax2.xaxis.tick_top()
    ax2.xaxis.set_label_position('top')

    # N-D
    ax3 = fig.add_subplot(1,5,3)
    ax3.plot(den, den.index, 'r', lw=1)
    ax3.xaxis.set_label_position('top')
    ax3.set_xlabel('ZDEN', labelpad=10, fontdict={'color':'r'})
    ax3.set_xlim(1.7, 2.7)
    ax3.invert_yaxis()
    ax3b = ax3.twiny()
    ax3b.plot(neut, neut.index, 'g', lw=1)
    ax3b.invert_xaxis()
    ax3b.set_xlabel('CNC', fontdict={'color':'g'})
    ax3b.set_xlim(0.6, 0.0)
    ax3b.xaxis.tick_top()
    ax3.xaxis.tick_top()
    ax3.set_xticks([1.7, 1.9, 2.1, 2.3, 2.5, 2.7])
    ax3.tick_params(axis='x', pad=35, top=True)
    ax3b.xaxis.set_label_position('top')
    ax3b.set_xticks([0.6, 0.48, 0.36, 0.24, 0.12, 0.0])

    # PE
    ax4 = fig.add_subplot(1,5,4)
    ax4.plot(pe, pe.index, 'k', lw=1)
    ax4.set_xlabel('PE')
    ax4.invert_yaxis()
    ax4.xaxis.tick_top()
    ax4.xaxis.set_label_position('top')

    # Sonic
    ax5 = fig.add_subplot(1,5,5)
    ax5.plot(dtc, dtc.index, 'b', lw=1)
    ax5.xaxis.set_label_position('top')
    ax5.set_xlabel('DTC', labelpad=10, fontdict={'color':'b'})
    ax5.set_xlim(50, 170)
    ax5.invert_xaxis()
    ax5.invert_yaxis()
    ax5b = ax5.twiny()
    ax5b.plot(dts, dts.index, 'r', lw=1)
    ax5b.invert_xaxis()
    ax5b.set_xlabel('DTS', fontdict={'color':'r'})
    ax5b.set_xlim(340, 100)
    ax5b.xaxis.tick_top()
    ax5.xaxis.tick_top()
    #ax5.set_xticks([170, 150, 130, 110, 90, 70, 50])
    ax5.set_xticks([230, 200, 170, 140, 110, 80, 50])
    ax5.tick_params(axis='x', pad=35, top=True)
    ax5b.xaxis.set_label_position('top')
    #ax5b.set_xticks([340, 300, 260, 220, 180, 140, 100])
    ax5b.set_xticks([460, 400, 340, 280, 220, 160, 100])

    plt.tight_layout()
    return fig

Helper function to plot RHOMAA-UMAA

In [ ]:
def plot_rhomaa_umaa(pe, dens, phi):
    """Plot log data using RHOMAA-UMAA crossplot for mineralogy
    
    Parameters:
    pe (ndarray-like) : PE curve
    dens (ndarray-like) : Density curve
    phi (ndarray-like) : Neutron porosity curve
    
    Returns:
    matplotlib.figure.Figure
    """
    # First, create the endmember points as a dataframe
    index = ['Quartz', 'Calcite', 'Dolomite', 'Anhydrite', 'K-Feldspar']
    data = {
        'rhoma' : [2.65, 2.71, 2.87, 2.95, 2.54],
        'uma' : [4.82, 13.79, 8.98, 14.99, 7.29],
    }
    df = pd.DataFrame(data=data, index=index)
    
    # Second, calculate umaa & rhomaa for the input logs
    u = pe * dens
    phid = (2.71 - dens)/(2.71-1.)
    phit = (phid + phi)/2
    #umaa = u / (1 - phit)
    #umaa = (u - phit*0.5)/(1-phit)/2
    rhomaa = (dens - phit*1.)/(1-phit) # assume brackish pore fluid rhof = 1.1 g/cc
    umaa = (pe * rhomaa) / 2
    
    # Finally, create and return figure
    fig = plt.figure(figsize=(28,10))
    
    gs = fig.add_gridspec(1, 6)
    
    ax1 = fig.add_subplot(gs[0,:-3])
    im = ax1.scatter(umaa, rhomaa, s=10, c=umaa.index, marker='.', cmap='inferno', alpha=0.8)
    ax1.scatter(df['uma'], df['rhoma'], c='r', marker='D')
    for i, txt in enumerate(df.index.tolist()):
        ax1.annotate(txt, (df['uma'].iloc[i]+0.03, df['rhoma'].iloc[i]-0.02), weight='bold')
    ax1.plot(df['uma'].loc[['Quartz','Calcite']], df['rhoma'].loc[['Quartz', 'Calcite']], 'k')
    ax1.plot(df['uma'].loc[['Quartz', 'Dolomite']], df['rhoma'].loc[['Quartz', 'Dolomite']], 'k')
    ax1.plot(df['uma'].loc[['Dolomite','Calcite']], df['rhoma'].loc[['Dolomite','Calcite']], 'k')
    ax1.set_xlabel('UMAA (barns/cc)')
    #ax1.set_xlim(2,16)
    ax1.set_ylabel('RHOMAA (g/cc)')
    #ax1.set_ylim(2.3, 3.1)
    ax1.invert_yaxis()
    ax1.set_title('RHOMAA-UMAA Crossplot', fontdict={'fontsize':20, 'fontweight':'bold'})
    cbar = fig.colorbar(im, ax=ax1)
    cbar.ax.invert_yaxis()
    cbar.set_label('sample no', fontdict={'fontweight':'bold'})
    
    ax4 = fig.add_subplot(gs[0,3])
    ax4.plot(phit, phit.index, 'k', lw=2, label='phit')
    ax4.plot(phi, phi.index, 'r', lw=0.5, label='phin')
    ax4.plot(phid, phid.index, 'g', lw=0.5, label='phid')
    ax4.invert_yaxis()
    ax4.set_ylabel('sample no')
    ax4.invert_xaxis()
    ax4.set_xlabel('Phi')
    ax4.xaxis.tick_top()
    ax4.xaxis.set_label_position('top')
    ax4.legend(loc='best')
    
    ax5 = fig.add_subplot(gs[0,4], yticklabels=())
    ax5.plot(umaa, umaa.index)
    ax5.invert_yaxis()
    ax5.set_xlabel('UMAA')
    ax5.xaxis.tick_top()
    ax5.xaxis.set_label_position('top')
    
    ax6 = fig.add_subplot(gs[0,5], yticklabels=())
    ax6.plot(rhomaa, rhomaa.index)
    ax6.invert_yaxis()
    ax6.set_xlabel('RHOMAA')
    ax6.xaxis.tick_top()
    ax6.xaxis.set_label_position('top')
    
    return fig
        
    

Helper function to plot Training Real vs. Training Predicted

In [ ]:
def plot_real_vs_pred(y, y_pred, feat_name, split):
    """Function that plots y vs y_pred
    
    Parameters:
    y (ndarray like) : real values
    y_pred (ndarray like) : predicted values
    feat_name (str) : Feature name
    split (str) : Whether this is 'Train', 'Test', or 'Full Log'
    
    returns matplotlib.figure.Figure
    """
    fig = plt.figure(figsize=(20,6))
    fig.suptitle('Real {} vs. Predict'.format(split))
    
    ax1 = fig.add_subplot(1,2,1)
    ax1.plot(y.reset_index(drop=True, inplace=False), 'k', lw=2, label='y {}'.format(split))
    ax1.plot(y_pred, 'r', lw=1, label='y_{}_pred'.format(split))
    ax1.set_xlabel('sample no.')
    ax1.set_ylabel(feat_name)
    ax1.set_title('{} Real & Predict'.format(split))
    ax1.legend(loc='best')
    
    ax2 = fig.add_subplot(1,2,2)
    ax2.scatter(y, y_pred)
    ax2.set_xlabel('{} Real {}'.format(feat_name, split))
    ax2.set_ylabel('{} Predict {}'.format(feat_name, split))
    ax2.set_title('{} {} Predict vs. Real'.format(feat_name, split))
    
    return fig

## 2. <a name="read-data"></a>Read in Data

Back to [top](#top)

In [ ]:
df_w1 = pd.read_csv('../data/train.csv')

In [ ]:
df_w1.head()

## 3. <a name="inspect-data"></a>Inspect Data

Back to [top](#top)

In [ ]:
df_w1.info()

All data are type float64

In [ ]:
df_w1.describe().transpose()

In [ ]:
plot_data_distributions(df_w1)
plt.show()

In [ ]:
plot_data_cdf(df_w1)
plt.show()

Samples with value of -999.0 need to be replaced with NaN

In [ ]:
df_w1.replace(to_replace=-999.0, value=np.nan, inplace=True)

In [ ]:
df_w1.describe().transpose()

In [ ]:
plot_data_distributions(df_w1)
plt.show()

In [ ]:
plot_data_cdf(df_w1)
plt.show()

There are some suspicious values here that we will explore in the next step.

1. CNC should range from 0.0 to 1.0
1. GR should not have values less than 0.0
1. PE should not have values less than 0.0
1. ZDEN should not have values less than 0.0

## <a name="eda"></a>4. Exploratory Data Analysis (EDA)

Back to [top](#top)

Before making any edits to values, plot the data for visual QC inspection

In [ ]:
well_plot_fig = plot_well_curves(df_w1)
plt.show()

Let's try a [RHOMAA-UMAA crossplot](http://www.kgs.ku.edu/Publications/Bulletins/LA/11_crossplot.html) to see if we can differentiate lithologies.

In [ ]:
plot_rhomaa_umaa(df_w1['PE'], df_w1['ZDEN'], df_w1['CNC'])
plt.show()

Definitely outliers are present!

For fun, lets see what a Short-Time Fourier Transform (STFT) of the well logs look like...

In [ ]:
stft_plot = plot_stft_well_curves(df_w1.interpolate(limit_area='inside'))
plt.show()

In [ ]:
df_w1.describe().transpose()

Limit CNC to range between 0.0 and 1.0

In [ ]:
df_w1.loc[df_w1['CNC'] < 0.0, ['CNC']] = np.nan
df_w1.loc[df_w1['CNC'] > 1.0, ['CNC']] = np.nan

Limit GR to range between 0.0 and 300.0

In [ ]:
df_w1.loc[df_w1['GR'] < 0.0, ['GR']] = np.nan
df_w1.loc[df_w1['GR'] > 300.0, ['GR']] = np.nan

Limit PE to have values no less than 0.0

In [ ]:
df_w1.loc[df_w1['PE'] < 0.0, ['PE']] = np.nan

Limit ZDEN to have values no less than 0.0

In [ ]:
df_w1.loc[df_w1['ZDEN'] < 0.0, ['ZDEN']] = np.nan

In [ ]:
df_w1.describe().transpose()

In [ ]:
plot_data_distributions(df_w1)
plt.show()

In [ ]:
plot_data_cdf(df_w1)
plt.show()

These value ranges are substantially more acceptable.  Re-plot the data and visually inspect

In [ ]:
well_plot_fig = plot_well_curves(df_w1)
plt.show()

Overall, this appears to be better.  The PE final 10,000 values look suspect, though.

Let's attempt filtering out the spikes remaining in the data.

In [ ]:
df_clean, df_outliers = filter_curves(df_w1, 27, 2)

In [ ]:
df_w1.describe().transpose()

In [ ]:
df_clean.describe().transpose()

In [ ]:
#cleaned_fig = plot_well_curves(df_clean)
#cleaned_fig = plot_well_curves(df_clean.fillna(method='bfill').fillna(method='ffill'))
cleaned_fig = plot_well_curves(df_clean.interpolate(limit_area='inside').dropna(axis=0, inplace=False))
#cleaned_fig = plot_well_curves(df_clean.dropna(axis=0, inplace=False).interpolate(limit_area='inside'))
#cleaned_fig = plot_well_curves(df_clean.dropna(axis=0, inplace=False))
plt.show()

In [ ]:
#qc_fig = qc_curve_filt(df_w1, df_clean)
qc_fig = qc_curve_filt(df_w1, df_clean.interpolate(limit_area='inside').dropna(axis=0, inplace=False))
#qc_fig = qc_curve_filt(df_w1, df_clean, df_outliers)
plt.show()

In [ ]:
plot_rhomaa_umaa(df_clean['PE'].interpolate(limit_area='inside'), df_clean['ZDEN'].interpolate(limit_area='inside'), df_clean['CNC'].interpolate(limit_area='inside'))
plt.show()

The filtering operation has done a good job of removing outliers while keeping the overall signal intact

PE definitely has some bad values, as evident both on the log plot and the RHOMAA-UMAA transform crossplot.

**CAL seems a little suspect, though.  It seems unusual for CAL to increase at the bottom of the hole**

In [ ]:
df_clean.interpolate(limit_area='inside').dropna(axis=0, inplace=False).describe().transpose()

There are several ways to proceed.

A few observations:

* CAL could be useful (variations in the wellbore are related to geomechanical properties), but it can also be rather stationary.
* If we want to maximize the amount of data available for estimating DTC & DTS, then interpolation is necessary.
* The last 10,000 samples in the PE curve appear to be bogus.  Maybe these can be estimated using the shallower section?
* HRM (Medium Resisitivity) should be dropped.  Without knowing the depth of investigation, it is likely to be contaminated with well-bore fluids.  HRD (Deep Resisitivity) is the better resistivity curve to use.  Maybe we could calculate Rt?

In [ ]:
plt.rcParams["axes.labelsize"] = 20

In [ ]:
sns.pairplot(df_clean.interpolate(limit_area='inside').dropna(axis=0, inplace=False))
plt.show()

Several predictors exhibit a degree of multicollinearity

* CNC & GR, which also exhibits heteroscedasticity
* CNC & ZDEN
* HRM & HRD

Feature ranking will be an important step

PCA may also be useful

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = sns.heatmap(df_clean.interpolate(limit_area='inside').dropna(axis=0, inplace=False).corr(), cmap="RdBu_r", annot=True, fmt=".2f")
ax.set_title('Correlation Coefficient Heatmap of Well Log Curves')
plt.show()

Observations:

* CAL has reasonably high positive correlation to both DTC & DTS, as well as CNC.
* CNC has reasonably high positive correlation to CAL, GR, PE, as well as to DTC & DTS
* GR has moderate positive correlation to DTC & DTS
* HRD & HRM do not exhibit strong correlation with any other parameters than themselves, *but this could be due to the fact they are log-scale*
* PE has moderately positive correlation with CNC, DTC, & DTS
* ZDEN has a high negative correlation with CNC, DTC, & DTS, and mild negative correlation with CAL, GR, & PE

The recommendation going forward will be to drop HRM from the set of inputs.

Make a copy of df_clean to take log transform of HRD & HRM to examine how they correlate to the other features

In [ ]:
df_clean_v2 = df_clean.copy()

In [ ]:
df_clean_v2['HRD'] = df_clean_v2['HRD'].apply(np.log)

In [ ]:
df_clean_v2['HRM'] = df_clean_v2['HRD'].apply(np.log)

In [ ]:
sns.pairplot(df_clean_v2.interpolate(limit_area='inside').dropna(axis=0, inplace=False))
plt.show()

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = sns.heatmap(df_clean_v2.interpolate(limit_area='inside').dropna(axis=0, inplace=False).corr(), cmap="RdBu_r", annot=True, fmt=".2f")
ax.set_title('Corr Coef Heatmap - log(HRx)')
plt.show()

Subset the data dropping HRM as it provides duplicate data

In [ ]:
df_subset = df_clean.copy().drop(labels='HRM', axis=1)

In [ ]:
df_subset.interpolate(limit_area='inside').dropna(axis=0, inplace=False).info()

In [ ]:
plot_data_distributions(df_subset)
plt.show()

In [ ]:
plot_data_cdf(df_subset)
plt.show()

This is coincidentally a great example of lognormal-ness in a natural system

In [ ]:
df_subset_interp = df_subset.copy().interpolate(limit_area='inside').dropna(axis=0)

In [ ]:
df_subset_interp.reset_index(drop=True, inplace=True)

In [ ]:
df_subset_interp.isnull().values.any()

In [ ]:
plot_data_distributions(df_subset_interp)
plt.show()

In [ ]:
plot_data_cdf(df_subset_interp)
plt.show()

In [ ]:
all_features = df_subset_interp.columns.tolist()
predictors = [x for x in all_features if x not in ['DTC', 'DTS']]

In [ ]:
for col in predictors:
    ax = compare_curves(df_subset_interp[col], df_subset_interp['DTC'], df_subset_interp['DTS'])
#plt.savefig('feature_compare_to_sonic_curves.png', dpi=100)
plt.show()

**Gameplan:**

1. Estimate missing PE for samples beyond 20000 and splice in
1. Estimate missing DTC & DTS using either Faust or Gardner and splice in
1. Build a data-driven model for estimating DTC & DTS

## <a name="fe"></a> 5. Feature Engineering

Back to [top](#top)

Before proceeding to estimating DTC & DTS, if we want to use as much of the data as possible, we need to attempt to estimate the bad values for PE as well as missing DTC & DTS.

1. [5a. PE Curve Estimation](#fe-pe)
1. [5b. DTC Approximation](#fe-dtc)
1. [5c. DTS Approximation](#fe-dts)

### <a name="fe-pe"></a> 5a. Feature Engineering - PE Curve Estimation

Back to [top](#top)

The goal is to estimate PE for the final 10,000 samples that appear to have a bad measurement.

If we can robustly estimate the PE curve that appears to be usable, then we can splice in the estimated section for the bad data samples, keeping the good data.

We previously observed that CNC & ZDEN were most correlated to PE, so take a closer look.

In [ ]:
sns.pairplot(df_subset_interp[['CNC','PE','ZDEN']])

It is also worth crossplotting PE and HRD using a semilog scale to see if there is any relation not immediately noticeable in linear scale

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(1,1,1)
ax = sns.scatterplot(x='HRD', y='PE', data=df_subset_interp)
ax.set_xscale('log')
plt.show()

We need to identify at which sample the values become constantly bad.  Maybe one way to do this is to look at the gradient

In [ ]:
df_subset_interp[df_subset_interp['PE'] < 2]

A quick df filter operation suggests that the bad PE values start at index 19338.

In [ ]:
fig = plt.figure(figsize=(20,10))

ax1 = fig.add_subplot(2,2,1)
ax1.plot(df_subset_interp['PE'].index, np.gradient(df_subset_interp['PE']))
ax1.scatter(19338, df_subset_interp['PE'].iloc[19338], c='k', s=100)
ax1.set_xlabel('sample no.')
ax1.set_ylabel('Gradient of PE')
ax1.set_title('Gradient of PE vs. Sample No.')

ax2 = fig.add_subplot(2,2,2)
ax2.plot(range(19200, 20000), np.gradient(df_subset_interp['PE'])[19200:20000])
ax2.scatter(19338, df_subset_interp['PE'].iloc[19338], c='k', s=100)
ax2.set_xlabel('sample no.')
ax2.set_ylabel('Gradient of PE')
ax2.set_title('Zoomed plot about PE Gradient Samples')

ax3 = fig.add_subplot(2,2,3)
ax3.plot(np.gradient(np.gradient(df_subset_interp['PE'])))
ax3.set_xlabel('sample no.')
ax3.set_ylabel('Second derivative of PE')
ax3.set_title('Second derivative of PE')

ax4 = fig.add_subplot(2,2,4)
ax4.plot(range(19200, 20000), np.gradient(np.gradient(df_subset_interp['PE']))[19200:20000], 'k', label='Second derivative')
ax4.plot(range(19200, 20000), np.gradient(df_subset_interp['PE'])[19200:20000], 'r', label='First derivative')
ax4.scatter(19338, df_subset_interp['PE'].iloc[19338], s=100)
ax4.set_xlabel('Sample no.')
ax4.set_ylabel('Second Derivative of PE')
ax4.set_title('Second Derivative of PE')

plt.legend(loc='best')
plt.tight_layout()
plt.show()

Visual inspection of the first and second derivative confirm that index 19338 is the start of the offending data.

Index Number 19338 is the point at which PE values appear to go consistently bad.  We can use all data up to this point for attempting to estimate PE.

In [ ]:
df_pe = df_subset_interp[['CNC','HRD','ZDEN','PE']].iloc[0:19338].copy()

Take the log of 'HRD' to put it in the same scale as CNC, ZDEN, & PE

In [ ]:
df_pe['HRD'] = df_pe['HRD'].apply(np.log)

In [ ]:
df_pe.rename(columns={'HRD':'log_HRD'}, inplace=True)

In [ ]:
df_pe.head()

In [ ]:
df_pe.info()

In [ ]:
df_pe.describe().transpose()

In [ ]:
#np.log(df_pe['HRD']).plot()
df_pe['log_HRD'].plot()
df_pe['PE'].plot()

In [ ]:
sns.pairplot(df_pe)

1. Train-Test-Split
1. Randomized Parameter Search + Cross Validation (Hyperparameter Tuning)
1. Grid-Search + K-Fold Cross Validation (Hyperparameter Fine Tuning)
1. Model Evaluation
1. QC real vs. predicted PE
1. Splice predicted data in for bad data, keeping good data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_pe[['CNC', 'log_HRD', 'ZDEN']], df_pe['PE'], test_size=0.3, random_state=random_state)

Instantiate a default Random Forest Regressor

In [ ]:
rf = RandomForestRegressor(random_state=random_state)

In [ ]:
print('Parameters currently in use:\n')
pprint(rf.get_params())

Define a random grid for RandomizedSearchCV to select initial hyperparameter values

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start=100, stop=1000, num=10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(start=10, stop=100, num=10)]

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

In [ ]:
random_grid = {'n_estimators': n_estimators,
              'max_features': max_features,
              'max_depth': max_depth,
              'min_samples_split': min_samples_split,
              'min_samples_leaf': min_samples_leaf,
              }

In [ ]:
pprint(random_grid)

Train the random search

In [ ]:
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=100, cv=5, verbose = 2, random_state=random_state, n_jobs=-1)

In [ ]:
rf_random.fit(X_train, y_train)

In [ ]:
rf_random.best_params_

Training Root Mean Squared Error for RandomForestRegressor with RandomizedSearchCV

In [ ]:
y_train_pred = rf_random.best_estimator_.predict(X_train)
print('Training RMSE: {}'.format(mean_squared_error(y_train, y_train_pred, squared=False)))
print('R2 Score: {}'.format(r2_score(y_train, y_train_pred)))

Testing Root Mean Squared Error for RandomForestRegressor with RandomizedSearchCV

In [ ]:
y_test_pred = rf_random.best_estimator_.predict(X_test)
print('Testing RMSE: {}'.format(mean_squared_error(y_test, y_test_pred, squared=False)))
print('R2 Score: {}'.format(r2_score(y_test, y_test_pred)))

Compare to a non-tuned default RandomForestRegressor

In [ ]:
rf_default = RandomForestRegressor(random_state=random_state)

In [ ]:
rf_default.fit(X_train, y_train)

Default RFR Training Root Mean Squared Error

In [ ]:
y_train_default_pred = rf_default.predict(X_train)
print('Default RF Training RMSE: {}'.format(mean_squared_error(y_train, y_train_default_pred, squared=False)))
print('Default RF R2 Score :{}'.format(r2_score(y_train, y_train_default_pred)))

Default RFR Testing Root Mean Squared Error

In [ ]:
y_test_default_pred = rf_default.predict(X_test)
print('Default RF Testing RMSE: {}'.format(mean_squared_error(y_test, y_test_default_pred, squared=False)))
print('Default RF R2 Score: {}'.format(r2_score(y_test, y_test_default_pred)))

Initial hyperparameter tuning using RandomizedSearchCV has improved the scores, but let's now use GridSearchCV to attempt to tune for a little more performance

In [ ]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [40, 45, 50, 55, 60],
    'max_features': ['sqrt'],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    'n_estimators': [int(x) for x in np.linspace(start=700, stop=900, num=21)],
}

In [ ]:
pprint(param_grid)

In [ ]:
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

GridSearchCV Train RMSE

In [ ]:
y_train_gs_pred = grid_search.best_estimator_.predict(X_train)
print('RF GridSearch Best Estimator RMSE: {}'.format(mean_squared_error(y_train, y_train_gs_pred, squared=False)))
print('RF GridSearch Best Estimator R2: {}'.format(r2_score(y_train, y_train_gs_pred)))

GridSearchCV Test RMSE

In [ ]:
y_test_gs_pred = grid_search.best_estimator_.predict(X_test)
print('RF GridSearch Best Estimator Test RMSE: {}'.format(mean_squared_error(y_test, y_test_gs_pred, squared=False)))
print('RF GridSearch Best Estimator Test R2: {}'.format(r2_score(y_test, y_test_gs_pred)))

The best estimator from gridsearch performed slightly worse on the training data but slightly better on the testing data.

In [ ]:
grid_search.best_estimator_.feature_importances_

Instantiate a new RandomForestRegressor using the parameters of the GridSearchCV best estimator

In [ ]:
rf_best = grid_search.best_estimator_

In [ ]:
rf_best.get_params

Use Yellowbrick to evaluate model performance further

In [ ]:
viz = FeatureImportances(rf_best)
viz.fit(X_train, y_train)
viz.show()

Plot Training vs. Predict & Test vs. Predict

In [ ]:
plot_real_vs_pred(y_train, y_train_gs_pred, feat_name='PE', split='Train')
plt.show()

In [ ]:
plot_real_vs_pred(y_test, y_test_gs_pred, feat_name='PE', split='Test')
plt.show()

Plot the entire curve using the model to compare performance

In [ ]:
y_whole_gs_pred = rf_best.predict(df_pe[['CNC', 'log_HRD', 'ZDEN']])

In [ ]:
plot_real_vs_pred(df_pe['PE'], y_whole_gs_pred, feat_name='PE', split='Full Log')
plt.show()

Not too shabby for the clean section of the log!

How does the entire predicted log look?

In [ ]:
df_pe_pred = pd.DataFrame()
df_pe_pred['CNC'] = df_subset_interp['CNC']
df_pe_pred['log_HRD'] = np.log(df_subset_interp['HRD'])
df_pe_pred['ZDEN'] = df_subset_interp['ZDEN']

In [ ]:
df_pe_pred['PE_est'] = rf_best.predict(df_pe_pred)

In [ ]:
plot_real_vs_pred(df_subset_interp['PE'], df_pe_pred['PE_est'], feat_name='PE', split='Full Log')
plt.show()

Now that we have what appears to be a robust estimate of the missing PE data, we can splice it in to the original data

In [ ]:
df_subset_interp['PE_est'] = df_subset_interp['PE']
df_subset_interp['PE_est'].iloc[19338:] = df_pe_pred['PE_est'].iloc[19338:]
df_subset_interp['log_HRD'] = df_subset_interp['HRD'].apply(np.log)

In [ ]:
plot_real_vs_pred(df_subset_interp['PE'], df_subset_interp['PE_est'], feat_name='PE', split='Full Log with Splicing')
plt.show()

In [ ]:
df_subset_interp.head()

In [ ]:
df_subset_interp.tail()

Check RHOMAA-UMAA crossplot with predicted PE values.

In [ ]:
plot_rhomaa_umaa(df_subset_interp['PE_est'], df_subset_interp['ZDEN'], df_subset_interp['CNC'])
plt.show()

In [ ]:
sns.pairplot(df_subset_interp.drop(labels='PE', axis=1, inplace=False))
plt.show()

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = sns.heatmap(df_subset_interp.drop(labels="PE", axis=1, inplace=False).corr(), cmap="RdBu_r", annot=True, fmt=".2f")
ax.set_title('Corr Coef Heatmap')
plt.show()

### <a name="fe-dtc"></a> 5b. Feature Engineering - DTC Approximation

back to [Feature Engineering](#fe)

back to [top](#top)

Next, we will attempt to approximate the missing DTC values where previously we have used a linear interpolation method.  There are several ways this could be accomplished.

1. Use [Gardner's Equation](#gardner) (1974) : $\rho = 0.23 * (V_{p}^{0.25})$, which can be re-written as $V_{p} = \sqrt[0.25]{\frac{\rho}{0.23}}$
1. Use Faust's Relationship (1951): $V_{p} = \alpha*(R*Z)^{\frac{1}{6}}$ where,

    $\alpha \equiv$ a depth-related constant ranging from 1000-3400,
    
    $R \equiv$ Resistivity, and
    
    $Z \equiv$ Depth in feet
    
    *Unfortunately, Faust cannot be used here because we don't know actual depth, only sample number*
    
1. Use a [model approach](#dtc-model) using another curve(s) which tracks DTC closely.  For example, CNC correlates very highly with DTC.

Recall that $V_{p} = \frac{1E6}{DTC}$, and $DTC = \frac{1E6}{V_{p}}$


#### <a name="gardner"></a> 5b.1 Gardner's Equation for estimating DTC

In [ ]:
Vp_gard = df_subset_interp['ZDEN'].apply(lambda x: np.power((x/0.23), 1/0.25))

In [ ]:
Vp_raw = 1e6/df_subset_interp['DTC']

In [ ]:
fig = plt.figure(figsize=(20, 6))
ax = fig.add_subplot(111)
ax.plot(Vp_raw, 'k', lw=2, label='Vp_raw')
ax.plot(Vp_gard, 'r', lw=1, label='Vp_gard')
ax.set_xlabel('Sample no.')
ax.set_ylabel('Vp')
ax.set_title('Vp vs. Vp from Density using Gardner\'s Equation')
ax.legend(loc='best')
plt.show()

In [ ]:
print('Root Mean Square Error of Gardner Equation prediction using ZDEN: {}'.format(evaluate_rmse(Vp_raw, Vp_gard)))

#### <a name="dtc-model"></a> 5b.2 Model approach

In [ ]:
fig = plt.figure(figsize=(30,6))

ax1 = fig.add_subplot(1,2,1)
ax1.plot(df_subset_interp['DTC'], 'k', lw=2, label='DTC')
ax1b = ax1.twinx()
ax1b.plot(df_subset_interp['CNC'], 'r', lw=1, label='CNC')
ax1.set_xlabel('Sample no.')
ax1.set_ylabel('DTC')
ax1b.set_ylabel('CNC', color='red')

ax2 = fig.add_subplot(1,2,2)
ax2.scatter(df_subset_interp['CNC'], df_subset_interp['DTC'])
ax2.set_xlabel('CNC')
ax2.set_ylabel('DTC')

plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.array(df_subset_interp['CNC']).reshape(-1,1), np.array(df_subset_interp['DTC']).reshape(-1,1), test_size=0.3, random_state=random_state)

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
print('LR coeff: {}'.format(lr.coef_))
print('LR intercept: {}'.format(lr.intercept_))

In [ ]:
print('LR training score: {}'.format(lr.score(X_train, y_train)))
print('LR test score :{}'.format(lr.score(X_test, y_test)))

In [ ]:
DTC_cnc = lr.predict(np.array(df_subset_interp['CNC']).reshape(-1,1))

In [ ]:
DTC_cnc.shape

In [ ]:
df_subset_interp['DTC'].shape

In [ ]:
plot_real_vs_pred(df_subset_interp['DTC'], DTC_cnc, feat_name='DTC', split='Full Log')
plt.show()

In [ ]:
print('RMSE predicting DTC from CNC: {}'.format(evaluate_rmse(df_subset_interp['DTC'], DTC_cnc[:,0])))

**This result is significantly more accurate compared to using the Gardner equation**

In [ ]:
fig = plt.figure(figsize=(25,10))

ax1 = fig.add_subplot(1,1,1)
ax1.plot(df_subset_interp['DTC'], 'k', lw=3, label='DTC_raw')
ax1.plot((1e6/Vp_gard), 'b', lw=1, label='DTC_gard')
ax1.plot((DTC_cnc+rng.normal(0,1,DTC_cnc.shape)), '--g', lw=1, label='DTC_cnc+random noise')
ax1.plot(DTC_cnc, 'r', lw=1, label='DTC_cnc')
ax1.legend(loc='best')
ax1.set_xlabel('sample no.')
ax1.set_ylabel('DTC')
ax1.set_title('Comparing raw DTC vs. two prediction methods')

plt.show()

**Linear Regression for DTC on CNC is a better estimator and should be used for the missing section.  Adding in some random noise is probably a good idea so it is not directly related to CNC**

*Alternatively, each could be used as as separate final models... One could be interpolated values, One could be DTC_gard, and One could be DTC_cnc*

**Now we need to splice in the DTC_cnc estimate for the zone that is interpolated**

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111)
ax.plot(df_subset_interp['DTC'], 'k', lw=2, label='DTC')
ax.plot(np.round(np.gradient(df_subset_interp['DTC']), 1), 'r', lw=1, label='gradient')
ax.plot(df_subset_interp['DTC'].diff(), 'b', lw=0.25, label='diff')
ax.set_xlabel('sample no.')
ax.set_ylabel('DTC')
ax.set_title('Gradient of DTC')
ax.legend(loc='best')
plt.show()

In [ ]:
df_subset_interp[np.abs(np.round(np.gradient(df_subset_interp['DTC']), 1)) == 0 ].loc[8441:12551,'DTC'].index

In [ ]:
fig = plt.figure(figsize=(20,6))
ax = fig.add_subplot(111)
ax.plot(df_subset_interp['DTC'], 'k', lw=2, label='DTC')
ax.plot(df_subset_interp['DTC'].iloc[8441:12551], 'r', lw=2, label='DTC interp')
ax.plot(np.round(np.gradient(df_subset_interp['DTC']),1), 'r', lw=1, label='gradient')
ax.scatter(df_subset_interp[np.abs(np.round(np.gradient(df_subset_interp['DTC']), 1)) == 0].loc[8441:12551,'DTC'].index, np.zeros(4108), s=20, c='b', marker='.', label='interpolated data')
ax.set_xlabel('sample no')
ax.set_ylabel('DTC')
ax.legend(loc='best')
plt.show()

There is probably a more elegant way to find the index range of the interpolated data, but for now this works

In [ ]:
df_subset_interp['DTC_spliced'] = df_subset_interp['DTC']
DTC_cnc_awgn = DTC_cnc + rng.normal(0,1,DTC_cnc.shape)
df_subset_interp['DTC_spliced'].iloc[8441:12551] = DTC_cnc_awgn[8441:12551,0]

In [ ]:
plot_real_vs_pred(df_subset_interp['DTC'], df_subset_interp['DTC_spliced'], feat_name='DTC', split='Full Log with Splicing')
plt.show()

In [ ]:
plot_well_curves(df_subset_interp)
plt.show()

DTC_spliced looks like a plausible estimation for the missing data section.  Notice the slow velocity layer just above sample number 10,000.  The same slow down is seen on the DTS log.  Encouraging!

In [ ]:
petro_plot(df_subset_interp['GR'], df_subset_interp['HRD'], df_subset_interp['CNC'], df_subset_interp['ZDEN'], df_subset_interp['PE_est'], df_subset_interp['DTC_spliced'], df_subset_interp['DTS'])
plt.show()

### <a name="fe-dts"></a> 5c. Feature Engineering - DTS Approximation

back to [Feature Engineering](#fe)

back to [top](#top)

Finally, there is a missing section of DTS that needs to be approximated.

1. For self-consistency with DTC estimation, first build a [Linear Regression](#lr-dts) for DTS on to CNC.
1. <a name="empirical-dts"></a>Empirical Models
    * [Castagna *et al.* (1993)](#castagna-1993): 
    ** Water Saturated Sands: $V_{s} (km/s) = 0.804V_{p} - 0.856$
    ** Mudrock line: $V_{s} (km/s) = 0.862V_{p} - 1.172$
    * [Brocher (2005)](#brocher-2005): $V_{s} (km/s) = 0.7858 - 1.2344V_{p} + 0.7949V_{p}^2 - 0.1238V_{p}^3 + 0.0064V_{p}^4$
    * Castanga *et al.* (1985): 
    ** Shaley sands (Frio formation): $V_{s} (km/s) = 3.89 - 7.07{phi} - 2.04C$, (km/sec) where ${phi}$ is porosity & $C$ is clay fraction
    ** Mudrock: $V_{s} (km/s) = 0.862V_{p} - 1.172$
    
    
For more on empirical models for Vp, Vs, & more, see __The Rock Physics Handbook, 2e__ by Mavko *et al.* (2009).  It is always a handy reference in times of uncertainty.

In [ ]:
fig = plt.figure(figsize=(20,6))
ax = fig.add_subplot(111)
ax.plot( df_subset_interp['DTC_spliced'].index, (1e6/df_subset_interp['DTC_spliced'])/(1e6/df_subset_interp['DTS']), 'k')
ax.set_xlabel('sample no')
ax.set_ylabel('Vp/Vs ratio')
ax.set_title('Vp/Vs ratio', fontdict={'fontsize':20, 'fontweight':'bold'})
plt.show()

Vp/Vs ratio is anomalously high in the "shallow" section until about sample number 10,000 at which it averages about 1.7 - 1.8.

Compare CNC & DTS

In [ ]:
fig = plt.figure(figsize=(20,6))
ax = fig.add_subplot(111)
ax.plot(df_subset_interp['DTS'], 'k', lw=2, label='DTS')
ax1 = ax.twinx()
ax1.plot(df_subset_interp['CNC'], 'r', lw=1, label='CNC')
ax.set_xlabel('sample no.')
ax.set_ylabel('DTS')
ax1.set_ylabel('CNC', fontdict={'color':'r'})
plt.show()


Build a <a name="lr-dts"></a>Linear Regression model to predict DTS from CNC

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.array(df_subset_interp['CNC']).reshape(-1,1), np.array(df_subset_interp['DTS']).reshape(-1,1), test_size=0.3, random_state=random_state)

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
print('LR coeff: {}'.format(lr.coef_))
print('LR intercept: {}'.format(lr.intercept_))

In [ ]:
print('LR training score: {}'.format(lr.score(X_train, y_train)))
print('LR test score :{}'.format(lr.score(X_test, y_test)))

In [ ]:
DTS_cnc = lr.predict(np.array(df_subset_interp['CNC']).reshape(-1,1))

In [ ]:
plot_real_vs_pred(df_subset_interp['DTS'], DTS_cnc, feat_name='DTS', split='Full Log')
plt.show()

In [ ]:
print('RMSE predicting DTS from CNC: {}'.format(evaluate_rmse(df_subset_interp['DTS'], DTS_cnc[:,0])))

This is an underwhelming result.  Whereas CNC was a relatively robust predictorfor DTC, the relationship does not seem to hold as well for DTS.

Instead, there are several well-known [empirical models](#empirical-dts) which we can test to see if we can produce a better estimate of Vs for the interpolated section.

Let's first test the well-known and widely applied <a name="castagna-1993"></a>Castana *et al.* (1993):

* For water saturated sandstones: $V_{s} (km/s) = 0.804V_{p} - 0.856$, and

* The famous "Mudrock" line: $V_{s} (km/s) = 0.862V_{p} - 1.172$

In [ ]:
Vs_c93ss = 1000*(0.804*((1e6/df_subset_interp['DTC_spliced']/3.281)/1000) - 0.856)
Vs_c93mr = 1000*(0.862*((1e6/df_subset_interp['DTC_spliced']/3.281)/1000) - 1.172)

In [ ]:
fig = plt.figure(figsize=(20,6))
ax = fig.add_subplot(111)
ax.plot(df_subset_interp['DTS'], 'k', lw=3, label='DTS')
ax.plot(1e6/(3.281*Vs_c93ss), 'r', lw=1, label='Vs Castagna et al. (1993) Sandstone')
ax.plot(1e6/(3.281*Vs_c93mr), 'b', lw=1, label='Vs Castagna et al. (1993) Mudrock')
ax.set_xlabel('sample no.')
ax.set_ylabel('Vs (m/s)')
ax.set_title('Comparison of DTS_raw & Castagna et al. (1993)')
ax.legend(loc='best')
plt.show()

In [ ]:
print('RMSE predicting DTS from Castagna et al. 1993 sandstone: {}'.format(evaluate_rmse(df_subset_interp['DTS'], 1e6/(3.281*Vs_c93ss))))
print('RMSE predicting DTS from Castagna et al. 1993 mudrock: {}'.format(evaluate_rmse(df_subset_interp['DTS'], 1e6/(3.281*Vs_c93mr))))

* On the whole, this is not a drastically different from the linear regression of DTS onto CNC, but there are several reasons why.  The lithology is not simply all wet sandstone or all wet mudrock.  Using the wet sandstone line performs slightly worse than the CNC-model while the mudrock line performs slighty better.

* To improve the model fit, we would need to know about the lithology.  

* First, let's take a look at the Vp/Vs ratio using these two estimates

In [ ]:
fig = plt.figure(figsize=(20,6))
ax = fig.add_subplot(111)
ax.plot( df_subset_interp['DTC_spliced'].index, (1e6/df_subset_interp['DTC_spliced'])/(1e6/df_subset_interp['DTS']), 'k', lw=3, label='Vp/Vs raw')
ax.plot(df_subset_interp['DTC_spliced'].index, (1e6/df_subset_interp['DTC_spliced']/3.281)/Vs_c93ss, 'r', lw=1, label='Vp/Vs C93SS')
ax.plot(df_subset_interp['DTC_spliced'].index, (1e6/df_subset_interp['DTC_spliced']/3.281)/Vs_c93mr, 'b', lw=1, label='Vp/Vs C93MR')
ax.set_xlabel('sample no')
ax.set_ylabel('Vp/Vs ratio')
ax.set_title('Vp/Vs ratio', fontdict={'fontsize':20, 'fontweight':'bold'})
ax.legend(loc='best')
plt.show()

Castagna *et al.* (1993) also has relations for limestones.  As this is the Volve field in the Norwegian North Sea, that is entirely a possibility:

$V_{s} (km/s) = -0.055V_{p}^2 + 1.017V_{p} - 1.031$

In [ ]:
Vs_c93ls = 1000*(-0.055*np.power(1e6/df_subset_interp['DTC_spliced']/3.281/1000, 2) + 1.017*(1e6/df_subset_interp['DTC_spliced']/3.281/1000) - 1.031)

In [ ]:
fig = plt.figure(figsize=(24,8))
ax = fig.add_subplot(111)
ax.plot(df_subset_interp['DTS'], 'k', lw=3, label='DTS raw')
ax.plot(1e6/(3.281*Vs_c93ss), 'r', lw=1, label='DTS C93 Sandstone')
ax.plot(1e6/(3.281*Vs_c93mr), 'b', lw=1, label='DTS C93 Mudrock')
ax.plot(1e6/(3.218*Vs_c93ls), 'g', lw=1, label='DTS C93 Limestone')
ax.set_xlabel('sample no')
ax.set_ylabel('DTS')
ax.set_title('Comparison of DTS with Castagna et al. (1993) Mudrock & Limestone relation')
ax.legend(loc='best')
plt.show()

In [ ]:
print('RMSE predicting DTS from Castagna et al. 1993 limestone: {}'.format(evaluate_rmse(df_subset_interp['DTS'], 1e6/(3.281*Vs_c93ls))))

Interesting... while the overall RMSE of the limestone trend is about the same as the sandstone, there are some zones along the log that this is clearly a better fit, suggesting a substantial proportion of limestone

Now, lets try <a name="brocher-2005"></a>Brocher's (2005) relation for Vs from Vp:

$V_{s} (km/s) = 0.7858 - 1.2344V_{p} + 0.7949V_{p}^2 - 0.1238V_{p}^3 + 0.0064V_{p}^4$

In [ ]:
# for ease, let's make a variable for Vp
Vp_mps = 1e6/df_subset_interp['DTC_spliced']/3.281
Vp_kmps = Vp_mps / 1000

In [ ]:
Vs_brocher = 0.7858 - 1.2344*Vp_kmps + 0.7949*np.power(Vp_kmps, 2) - 0.1238*np.power(Vp_kmps, 3) + 0.0064*np.power(Vp_kmps, 4)

In [ ]:
fig = plt.figure(figsize=(20,6))
ax = fig.add_subplot(111)
ax.plot(df_subset_interp['DTS'], 'k', lw=3, label='DTS raw')
ax.plot(1e6/(Vs_brocher*1000*3.281), 'r', lw=1, label='DTS Brocher 2005')
ax.plot(1e6/(Vs_c93mr*3.281), '--b', lw=1, label='DTS Castagna et al. (1993) mudrock')
ax.set_xlabel('sample no')
ax.set_ylabel('DTS')
ax.set_title('Comparison of Castagna et al. (1993) Mudrock and Brocher (2005)')
ax.legend(loc='best')
plt.show()

In [ ]:
print('RMSE predicting DTS from Brocher (2005): {}'.format(evaluate_rmse(df_subset_interp['DTS'], 1e6/(Vs_brocher*1000*3.281))))

Brocher (2005) is almost identical to Castagna *et al.* (1993) Mudrock line!